In [36]:
import torch
import torch.nn as nn

class DenseBlock(nn.Module):
    def __init__(self, n_channel,norm="BatchNorm2d",dropout=0.0,activation="LeakyReLU",depth=5):
        super().__init__()

        self.depth = depth

        ## Conv 2d
        self.seq_conv = []
        for i in range(depth):
            module = nn.Conv2d(n_channel+i*n_channel,n_channel,kernel_size=(3,3),stride=(1,1),padding=(1,1))
            self.seq_conv.append(module)
            self.add_module("conv{}".format(i),module)

        ## Normalization
        bn_module = nn.InstanceNorm2d
        self.bn = bn_module(n_channel)

        self.acti = nn.LeakyReLU()
    def forward(self,x):

        skip = []
        y = self.seq_conv[0](x)
        skip.append(y)
        print(y.shape)

        for i in range(1,self.depth) : 
            y = self.seq_conv[i](torch.cat([x,skip]),dim=1)
            print(y.shape)
            skip.append(y)

        return y

In [37]:
x = torch.zeros(2,3)
l=[]
l.append(x)
l.append(x)
print(l)

l = torch.tensor(l)


[tensor([[0., 0., 0.],
        [0., 0., 0.]]), tensor([[0., 0., 0.],
        [0., 0., 0.]])]


ValueError: only one element tensors can be converted to Python scalars

In [31]:
x = torch.rand(1,32,127,125)
print(x.shape)
m = DenseBlock(32)
print(m)
y = m(x)
print(y.shape)

torch.Size([1, 32, 127, 125])
DenseBlock(
  (conv0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(160, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (acti): LeakyReLU(negative_slope=0.01)
)
torch.Size([1, 32, 127, 125])


TypeError: cat() received an invalid combination of arguments - got (Tensor, list), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)
